# Import the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

# Load the Dataset

In [2]:
df = pd.read_csv("covid.csv")

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

# Split the Dataset into Feature and Target

In [7]:
X = df.drop("has_covid", axis = 1)
y = df["has_covid"]

In [8]:
print(X.shape, y.shape)

(100, 5) (100,)


# Split the Dataset into Training and Testing Data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
print(X.shape, X_train.shape, X_test.shape)

(100, 5) (80, 5) (20, 5)


In [11]:
X_train.head()

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi


# Without Column Transformer

### Imputation

In [12]:
# Load the SimpleImputer Function
si = SimpleImputer()

# Training Data
X_train_fever = si.fit_transform(X_train[["fever"]])

# Testing Data
X_test_fever = si.transform(X_test[["fever"]])

In [14]:
print(X_train_fever.shape, X_test_fever.shape)

(80, 1) (20, 1)


### OrdinalEncoding

In [13]:
# Load the OrdinalEncoder Function
oe = OrdinalEncoder(categories = [["Mild", "Strong"]])

# Training Data
X_train_cough = oe.fit_transform(X_train[["cough"]])

# Testing Data
X_test_cough = oe.transform(X_test[["cough"]])

In [14]:
X_test_cough

array([[0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.]])

### OneHotEncoding

In [15]:
# Load the OneHotEncoder Function
ohe = OneHotEncoder(drop = "first", sparse_output = False)

# Training Data
X_train_gender_city = ohe.fit_transform(X_train[["gender", "city"]])

# Testing Data
X_test_gender_city = ohe.transform(X_test[["gender", "city"]])

In [16]:
X_test_gender_city

array([[0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.]])

# Concatenation 

### Extracting Age

In [20]:
# Training Data
X_train_age = X_train.drop(columns = ["gender", "fever", "cough", "city"]).to_numpy()

# Testing Data
X_test_age = X_test.drop(columns = ["gender", "fever", "cough", "city"]).to_numpy()

In [21]:
X_train_age.shape

(80, 1)

### Transform the Data

In [24]:
# Training Data
X_train_transformed = np.concatenate((X_train_age, X_train_gender_city, X_train_cough), axis = 1)

# Testing Data
X_test_transformed = np.concatenate((X_test_age ,X_test_fever, X_test_gender_city, X_test_cough), axis = 1)

In [26]:
X_test_transformed

array([[ 17., 104.,   0.,   0.,   1.,   0.,   0.],
       [ 83.,  98.,   1.,   1.,   0.,   0.,   0.],
       [ 68., 101.,   0.,   1.,   0.,   0.,   1.],
       [ 72.,  99.,   1.,   0.,   0.,   0.,   0.],
       [ 20., 102.,   1.,   1.,   0.,   0.,   1.],
       [ 50., 103.,   0.,   0.,   1.,   0.,   0.],
       [ 71.,  98.,   0.,   0.,   1.,   0.,   1.],
       [ 14.,  99.,   0.,   0.,   0.,   1.,   0.],
       [ 75., 101.,   0.,   1.,   0.,   0.,   0.],
       [ 60., 103.,   1.,   0.,   1.,   0.,   0.],
       [ 64.,  98.,   0.,   0.,   0.,   0.,   0.],
       [ 15., 101.,   1.,   1.,   0.,   0.,   0.],
       [ 34.,  98.,   1.,   0.,   1.,   0.,   1.],
       [ 26.,  98.,   0.,   0.,   1.,   0.,   0.],
       [ 59.,  99.,   0.,   1.,   0.,   0.,   1.],
       [ 65., 101.,   0.,   0.,   0.,   1.,   0.],
       [ 80., 100.,   1.,   0.,   0.,   0.,   0.],
       [  8., 101.,   0.,   0.,   1.,   0.,   0.],
       [ 25.,  99.,   0.,   0.,   1.,   0.,   1.],
       [ 83., 103.,   1.,   0.,

# With Column Transformer

In [27]:
from sklearn.compose import ColumnTransformer

In [29]:
transformer = ColumnTransformer(transformers = [
    ("tnf1", SimpleImputer(), ["fever"]),
    ("tnf2", OrdinalEncoder(categories = [["Mild", "Strong"]]), ["cough"]),
    ("tnf3", OneHotEncoder(sparse_output = False, drop = "first"), ["gender", "city"])
], remainder="passthrough")

In [30]:
transformer.fit_transform(X_train).shape

(80, 7)

In [31]:
transformer.transform(X_test).shape

(20, 7)